In [49]:
import pandas as pd

In [50]:
df: pd.DataFrame = pd.read_pickle('df.p')

df.reset_index(inplace=True)

In [51]:
df[df['url'] == 'https://api.github.com/repos/Automattic/themes/issues/5351']

,index,url,created_at,title,issue,comments,keywords
15945,51,https://api.github.com/repos/Automattic/themes...,2022-01-16 21:09:47,Shawburn: When setting Primary colour in Custo...,{'url': 'https://api.github.com/repos/Automatt...,[{'url': 'https://api.github.com/repos/Automat...,['cookie notice']


In [52]:
df['type'] = df['issue'].apply(lambda x: 'pr' if 'pull_request' in x else 'issue')

In [53]:
df['html_url'] = df['issue'].apply(lambda x: x['html_url'])

In [54]:
df['html_url']

0        https://github.com/duckduckgo/privacy-configur...
1        https://github.com/WaterfoxCo/Waterfox/issues/203
2         https://github.com/resonatecoop/stream/issues/93
3        https://github.com/brave/browser-laptop/issues...
4          https://github.com/duckduckgo/Android/pull/2665
                               ...                        
21603    https://github.com/EFForg/action-center-platfo...
21604    https://github.com/ualbertalib/jupiter/issues/508
21605    https://github.com/department-of-veterans-affa...
21606    https://github.com/cfpb/consumerfinance.gov/pu...
21607       https://github.com/GeoNode/geonode/issues/3035
Name: html_url, Length: 21608, dtype: object

In [55]:
len(df[df['type'] == 'pr']), len(df[df['type'] == 'issue'])

(4504, 17104)

In [56]:
df[(df['type'] == 'pr') & (df['html_url'].str.contains('issue'))]

,index,url,created_at,title,issue,comments,keywords,type,html_url


In [57]:
df.drop(df[df['type'] == 'pr'].index, inplace=True)
len(df[df['type'] == 'pr']), len(df[df['type'] == 'issue'])

(0, 17104)

In [58]:
df['project'] = df['issue'].apply(lambda x: '/'.join(x['repository_url'].split('/')[-2:]))

In [59]:
blacklisted_repositories = [
    'AdguardTeam/AdguardFilters',
    'vz-risk/VCDB'
]

df['blacklisted'] = df['project'].apply(lambda x: x in blacklisted_repositories)
len(df[df['blacklisted'] == True]), len(df[df['blacklisted'] == False])

(3828, 13276)

In [48]:
df.drop(df[df['blacklisted'] == True].index, inplace=True)
len(df[df['blacklisted'] == True]), len(df[df['blacklisted'] == False])

(0, 13276)

In [31]:

df['#comments'] = df['comments'].apply(lambda x: len(x))

# pandas get row with max value
df.loc[df['#comments'].idxmax()]


index                                                       526
url           https://api.github.com/repos/audacity/audacity...
created_at                                  2021-07-03 20:45:28
title            New privacy policy is completely unacceptable!
issue         {'url': 'https://api.github.com/repos/audacity...
comments      [{'url': 'https://api.github.com/repos/audacit...
keywords                                     ['privacy breach']
type                                                      issue
html_url       https://github.com/audacity/audacity/issues/1213
project                                       audacity/audacity
#comments                                                   635
Name: 12139, dtype: object

In [32]:
df['discussants'] = df['comments'].apply(lambda x: list(set([i['user']['login'] for i in x])))
df['#discussants'] = df['discussants'].apply(lambda x: len(x))

# pandas get row with max value
df.loc[df['#discussants'].idxmax()]

index                                                         191
url             https://api.github.com/repos/docker/docs/issue...
created_at                                    2018-06-20 07:19:51
title                       Download Docker CE without logging in
issue           {'url': 'https://api.github.com/repos/docker/d...
comments        [{'url': 'https://api.github.com/repos/docker/...
keywords        ['right to be forgotten', 'privacy breach', 'a...
type                                                        issue
html_url               https://github.com/docker/docs/issues/6910
project                                               docker/docs
#comments                                                     611
discussants     [anaisbetts, OliverCole, aral, raed667, Amazin...
#discussants                                                  359
Name: 1066, dtype: object

In [33]:
df['reporter'] = df['issue'].apply(lambda x: x['user']['login'])

In [34]:
df['reporting_date'] = df['issue'].apply(lambda x: x['created_at'])
df['reporting_date'] = pd.to_datetime(df['reporting_date'])
df['last_active_date'] = df['issue'].apply(lambda x: pd.to_datetime(x['updated_at']))
df['last_active_date'] = pd.to_datetime(df['last_active_date'])

In [35]:
df['status'] = df['issue'].apply(lambda x: x['state'])
df['labels'] = df['issue'].apply(lambda x: [label['name'] for label in x['labels']])
df['#labels'] = df['labels'].apply(lambda x: len(x))

In [36]:
df['issue_number'] = df['issue'].apply(lambda x: x['number'])

In [37]:
df[df['#labels'].notnull() & df['#labels'] > 0]['labels']

1                                        [feature_request]
3           [suggestion, feature/shields, feature/cookies]
8                                             [backburner]
16                                           [enhancement]
23                                                  [epic]
                               ...                        
21578                                                 [ui]
21580                                             [tested]
21583                                    [awaiting-review]
21590    [T: Anti Adblock Script, N: AdGuard for Window...
21601                         [question, privacy, Firefox]
Name: labels, Length: 7517, dtype: object

In [38]:
pd.to_pickle(df, 'issues.p')

In [39]:
df = pd.read_pickle('issues.p')

tmp_list = [[reporter + '_' + project] + [discussant + '_' + project for discussant in discussants] for project, reporter, discussants in zip(df['project'], df['reporter'], df['discussants'])]
users = [item for sublist in tmp_list for item in sublist]
print(len(users))
print(len(set(users)))


66928
41238


### -> now run the contributor import, then continue with `processing_contributor_import.ipynb`